# EEE466 Lab 2: Reliable UDP File Transfer

## Part 1: Background

The objective of this lab is for you to design and implement your own reliable communication protocol. You are tasked with creating a client-server application for file transfer. UDP-based distributed systems are becoming increasingly common. UDP offers some advantages over TCP including:
1. Reduced overhead: UDP's connectionless nature eliminates the overhead of establishing and maintaining connections found in TCP. This translates to lower latency and less resource consumption, crucial in real-time applications like video streaming or online gaming.
2. Greater control: UDP exposes the datagrams themselves to the application layer. This fine-grained access allows developers to design custom reliability mechanisms tailored to their application's specific needs. For instance, they can implement selective retransmission strategies or prioritize specific data packets based on their importance, potentially achieving more efficient error handling and recovery than TCP's one-size-fits-all approach.
3. Multicast/broadcast support: UDP's inherent support for multicast and broadcast communication makes it useful for scenarios where data needs to be distributed to multiple recipients efficiently.

In scenarios where the lightweight nature of UDP is desired but reliability is also necessary, reliability can be built into the application layer. This approach may involve implementing application-level acknowledgements to confirm message receipt, using sequence numbers to detect lost or out-of-order messages, and setting timeouts for retransmitting messages if no acknowledgement is received within a specified period. Error detection mechanisms, such as checksums, can be used to identify corrupted messages, and buffering can be employed to hold out-of-order messages until missing messages arrive to ensure proper sequencing.

### Reading and writing bytes

Python (and most high level programming languages) have an option to read raw bytes from a file. This is particularly useful for transmitting files such as images, videos, or binary data, but it can also be used for text files. Reading a file in bytes is done by opening the file in bytes mode, which is specified by adding a 'b' to the file mode in the open() function (e.g., `rb` for "read bytes"). Likewise, we can write byte data using `wb` or "write bytes". Reading and writing byte data in python is shown in the example below.

```py
# Define the source and destination file paths
source_path = 'source_file_path.bin'
destination_path = 'destination_file_path.bin'

with open(source_path, 'rb') as file:
    file_data = file.read()

with open(destination_path, 'wb') as file:
    file.write(file_data)
```

### Socket timeouts

Receiving a message is by default a blocking call in python sockets. If we create a new UDP socket and call `recvfrom()` with no one on the other end, it will block forever.  In designing distributed systems, it's often desirable to wait for a fixed window of time. If we don't receive the expected message, we want to move on to other logic or perhaps try again. The [settimeout method](https://docs.python.org/3/library/socket.html#socket.socket.settimeout) enables us to put an upper bound on the time that a blocking call will wait for. If no message is buffered within this window, a timeout error is raised. Below is an example of how we can handle these errors to perform useful logic in python.

In [ ]:
import socket

my_sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
my_sock.settimeout(1)

try:
    message_from_space, address = my_sock.recvfrom(1024)
    print(f"Aliens responded from {address} with a message: {message_from_space}")
except socket.timeout as e:
    print(f"Encountered the following error: {e}")
    print("...I guess the aliens aren't very chatty today")

## Part 2: Design

In this lab you'll use a generic interface to allow for *communicator objects* to be created dynamically within the context of the server and client code. These communicator objects are referred to as *stubs* in the context of distributed systems. The communicator interface, `CommInterface` is an abstract class that is inherited by the transfer interface `UDPFileTransfer`. The interface defines a contract for sending and receiving messages and files. By separating the communication logic from the core functionality of the client and server, the codebase becomes more modular, maintainable, and easier to extend or modify. Additionally, client and server code can interact with this interface without worrying about the underlying implementation. The `CommInterfaceFactory` class implements a factory pattern that instantiates a server stub or a client stub depending on its parameters.

Both client and server programs are provided as part of this lab. They already contain all the logic necessary to interface with `UDPFileTransfer`, including instantiating a stub. The client has a "work order" of tasks to request of the server, either PUT or GET commands to upload or download files, respectively. The command and parameters of each work order entry determines what method of `UDPFileTransfer` will be called via the client stub. Likewise, the server waits to receive messages from the client and parses them to determine what action to take. Consider the `ServerFS` directory and the `ClientFS` directory included with this lab as being part of separate file systems. In your `UDPFileTransfer` interface you will create methods that will allow either a client or server to read/write bytes to/from their filesystem. I.e., files in each entity's `upload` directory that will get transferred to the other's `download` directory. You will need to send and receive data from the files in chunks of bytes. Ensure that you have a good understanding of how the client and server work, you are not permitted to modify them.

### Important
Before starting your implementation, draft a plan for managing reliability. Recall how request-reply and request-reply-acknowledge protocols use different mechanics in order to handle duplicates, misordered packets, etc. Your plan should include a simple sequence diagram to describe how your protocol will handle different failure conditions. Present your plan to your instructor before the end of the first lab period.

### 1. Implement FTP
You can consider the problem in two parts: **file transfer** and **reliability**. As such, you probably will want to set the probability of network issues to 0 in order to work on each problem in sequence. Starting out, you'll just want to get the client and server sending and receiving messages, from there, build in the file data transfer. Here start with all of the network issues set to 0 and build the flow of the program, from command to a file transfer loop, to another command, etc. The transitions between tasks are very important, the sender and receiver must both agree on what information is being passed at each step of an operation.

### 2. Reliable Protocol Design
In this lab, three network issues are simulated: transmission latency, message loss, and duplicate messages. You must contend with a random chance of these issues occurring by building a custom reliability protocol into your application. Note that features such as error checking and flow control are not required.

TCP uses mechanisms such as **buffers** and **header information** to ensure reliability. One way to approach this problem is to adapt the solutions built into TCP for your application-level reliability. RFC (Request for Comment) are documents published by them the Internet Engineering Task Force (IETF) that describes specifications related to the operation, design, or implementation of internet technologies and are often a helpful resource. You may consider how to design your message headers by comparing the TCP header (https://www.rfc-editor.org/rfc/rfc9293#name-header-format) to the UDP header (https://www.rfc-editor.org/rfc/rfc768). It's easy to "over-engineer" your protocol so be intentional about what information your messages should include. Your message headers should be included in your implementation plan.

## Part 3: Requirements
This lab requires the following:

### 1. Reliable UDP Class (UDPFileTransfer)
- Inherits and implements all methods from CommInterface
- Does not use TCP
- Uses unmodified `_send()` method to perform all UDP packet transmissions
- Provides functions to read from and write to arbitrary paths
- Reads, writes, and transfers `.txt` and `.jpg` file types in raw bytes
- Message size is limited to 1024 bytes; larger files are broken into chunks and reassembled by the receiver
- Handles message parsing and I/O errors; prints custom error messages to console 
- Can use any sized header (less than 1024 bytes) to encode protocol information
- Can have any number of additional functions

### 2. Client and Server
The client and server applications are provided to you, you are not permitted to modify them. The client must be able to carry out the work order provided in the JSON file involving the successful PUT and GET of text and image data.

### 3. Reliability
To score full points on reliability, the application must be capable of sending files (client->server) and receiving files (server->client), including saving to disk without error, given the following settings simultaneously:
- Duplicate probability: 10%
- Drop probability: 10%
- Lag probability: 10%

## Part 4: Questions
1. Describe how you implemented an application level protocol to handle reliability and command and data messages. Include a sequence diagram in your submission. 
2. Describe how your solution implements *tiering* and *layered* architecture.
3. How does the parameter 'b' (bytes) change the way Python performs `read()` and `write()` actions?


## Part 5: Submission
Submit a zip archive in the format `eee446_lab2_lastname1_lastname2.zip` that includes the following:
1. A lab report notebook including a *brief* report detailing your design choices during this lab and your answers to the questions.
2. Your project folder including all python files and the work order json. Do not include the client and server filesystems.
3. A sequence diagram of your protocol design.